## Topic Modelling Gensim

In [1]:
import gensim
import gensim.corpora as corpora
import pickle
import pandas as pd
from pprint import pprint
from gensim.models import CoherenceModel
import csv

In [ ]:
# select tokens and subsample
tokenlist = pd.read_pickle("dataset_with_tokens.pkl")[["page", "tokens"]]
tokenlist = tokenlist["tokens"].tolist()

In [ ]:
# make bigram and trigram mod

# is that necessary?
bigram = gensim.models.Phrases(tokenlist, min_count = 5, threshold = 100)
trigram = gensim.models.Phrases(bigram[tokenlist], threshold = 100)

trigram_mod = gensim.models.phrases.Phraser(trigram)  
bigram_mod = gensim.models.phrases.Phraser(bigram)

tokenlist = [trigram_mod[bigram_mod[text]] for text in tokenlist]

In [ ]:
tokens_trigrams = []
for doc in tokenlist:
    tokenstring = [" ".join([token for token in doc])]
    tokens_trigrams.append(tokenstring)

In [ ]:
with open("tokens/tokens_trigrams.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerows(tokens_trigrams)

In [ ]:
# create dictionary (each word recorded once and connected to id)
id2word = corpora.Dictionary(tokenlist)
id2word.save("gensim_models/id2word")

# create corpus
corpus = [id2word.doc2bow(text) for text in tokenlist]
import pickle
with open("gensim_models/corpus", "wb") as f:
    pickle.dump(corpus, f)


In [2]:
# load dictionary
id2word = gensim.corpora.dictionary.Dictionary.load("gensim_models/id2word")

# load corpus
class MyCorpus:
    def __iter__(self):
        for line in open("tokens/tokens_trigrams.csv"):
            # assume there's one document per line, tokens separated by whitespace
            yield id2word.doc2bow(line.lower().split())

corpus = MyCorpus()


In [3]:
# build LDA model

lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           eval_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = "symmetric",
                                           per_word_topics = True,
                                           workers = 4)

In [7]:
lda_model.save("gensim_models/lda")

In [9]:
modell =  gensim.models.ldamodel.LdaModel.load("gensim_models/lda")

In [5]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("gensim_models/lda")
lda_model.save(temp_file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/hennes/.local/lib/python3.8/site-packages/gensim/test/test_data/gensim_models/lda.state'

In [4]:
# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [ ]:
# print word loadings on each topic
pprint(lda_model.print_topics())

In [ ]:
# not sure what this saves
doc_lda = lda_model[corpus]

### Model Performance

In [15]:
corrpus = (row for row in open("tokens/tokens_trigrams.csv"))

In [18]:
# Compute Perplexity
print('\nPerplexity: ', modell.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

TypeError: object of type 'MyCorpus' has no len()

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenlist, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### Visualising Model

In [21]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

with open("gensim_models/corpus", "rb") as f:
    corpus = pickle.load(f)

In [22]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(modell, corpus, id2word)
vis

/home/hennes/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.186443  0.052439       1        1  23.163968
2      0.014322  0.174002       2        1  13.288508
5     -0.033835  0.042394       3        1  11.870917
1     -0.069220 -0.155749       4        1  10.600330
9     -0.105432 -0.129556       5        1   9.115325
3      0.011174 -0.140342       6        1   7.624500
0      0.279819  0.147959       7        1   7.099552
8     -0.190548  0.169956       8        1   6.864767
7      0.169390 -0.090825       9        1   6.615615
4      0.110772 -0.070279      10        1   3.756517, topic_info=             Term           Freq          Total Category  logprob  loglift
285     nicaragua  222040.000000  222040.000000  Default  30.0000  30.0000
259         salud   78224.000000   78224.000000  Default  29.0000  29.0000
103       policía   66685.000000   66685.000000  Default  28.0000  28.0000
332        pueblo   61008.000000   61008.000000  Default  27.0000  27.0000
197       familia   91763.000000   91763.000000  Default  26.0000  26.0000
..            ...            ...            ...      ...      ...      ...
460  nicaragüense    3964.069664   78348.782326  Topic10  -5.0599   0.2978
219       atender    3071.993529   23271.000445  Topic10  -5.3148   1.2568
9       comunidad    3189.296773   38217.394880  Topic10  -5.2773   0.7982
377     seguridad    2769.271831   41960.858258  Topic10  -5.4186   0.5635
1             año    2713.765140  120218.520018  Topic10  -5.4388  -0.5093

[749 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
6451       7  0.999619  abogada
5816       2  0.999751  abogado
7572      10  0.998428   abonar
1355       1  0.252876    abril
1355       2  0.437801    abril
...      ...       ...      ...
1116       5  0.078975     zona
1116       7  0.000756     zona
1116       9  0.591429     zona
1116      10  0.251691     zona
13276      2  0.999275       és

[1890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 6, 2, 10, 4, 1, 9, 8, 5])

## Documents and Topics

In [ ]:
# find dominant model in each doc

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=tokenlist):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=tokenlist)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# find most representative document for each topic

# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# topic distribution across documents
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics